In [1]:
!pip install imblearn
!pip install psycopg2

  Using cached psycopg2-2.8.6-cp37-cp37m-win_amd64.whl (1.1 MB)


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC                     
from sklearn.metrics import plot_confusion_matrix, plot_confusion_matrix, roc_curve, roc_auc_score, f1_score, auc, precision_recall_curve, classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.combine import SMOTEENN, SMOTETomek
from sqlalchemy import create_engine

ImportError: cannot import name 'six'

In [3]:
login = 'postgres'
name = 'ccfraud.cz4qo4jmiwzd.us-west-2.rds.amazonaws.com'
password = 'd3rFE96Bh7XHtCHt6fyc'
db_name = 'ccfraud'
port = '5432'

rds_connection_string = f"{login}:{password}@{name}:{port}/{db_name}"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [4]:
credit_data = pd.read_sql_query('select * from synthetic', con=engine)

In [5]:
credit_data

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [ ]:
credit_data['newType']=credit_data['newType'].str.replace('CASH_IN','1')
credit_data['newType']=credit_data['newType'].str.replace('CASH_OUT','2')
credit_data['newType']=credit_data['newType'].str.replace('DEBIT','3')
credit_data['newType']=credit_data['newType'].str.replace('PAYMENT','4')
credit_data['newType']=credit_data['newType'].str.replace('TRANSFER','5')
credit_data['newType']=credit_data['newType'].astype(int)
credit_data

In [8]:
#Credit Card Data

#Create Data
X_C = credit_data.drop(['isFraud', "isFlaggedFraud",'nameOrig','nameDest','type'], axis=1).to_numpy()
y_C = credit_data['isFraud'].to_numpy()
del credit_data

#test-train split
X_train_C, X_test_C, y_train_C, y_test_C = train_test_split(X_C, y_C, random_state=1)

In [9]:
# Oversample:

#Oversample data
ros_C = RandomOverSampler(random_state=1)
X_resampled_C, y_resampled_C = ros_C.fit_resample(X_train_C, y_train_C)

In [10]:
#Log model for oversampled

#Create Log Model
model_log = LogisticRegression(random_state=1)
model_log.fit(X_resampled_C, y_resampled_C)

#Run Predictions
predictions_C = model_log.predict(X_test_C)

#Run Scoring methods
score_C = model_log.score(X_test_C, y_test_C)

probs = model_log.predict_proba(X_test_C)[:, 1]
fpr, tpr, _ = roc_curve(y_test_C, probs)

precision_C, recall_C, thresholds_C = precision_recall_curve(y_test_C, probs)
auc_C = auc(recall_C, precision_C)

f1_C = f1_score(y_test_C, predictions_C)

scores = ['score', 'auc', 'f1']
values = [score_C, auc_C, f1_C]
score_df = pd.DataFrame({'Scores': scores, 'Log Regression': values})

precision_recall_log_df = pd.DataFrame({'Precision_log': precision_C, 'Recall_log': recall_C})
roc_curve_log_df = pd.DataFrame({'fpr_log': fpr, 'tpr_log':tpr})
predictions_df = pd.DataFrame({'Actual_Results': y_test_C, 'Log_Results': predictions_C})

ValueError: could not convert string to float: 'CASH_IN'

In [ ]:
score_df

In [ ]:
# KNN for oversampled

from sklearn.preprocessing import StandardScaler

#Run standard scalar
X_scaler_C = StandardScaler().fit(X_resampled_C)

#Transform Data
X_train_scaled_C = X_scaler_C.transform(X_resampled_C)
X_test_scaled_C = X_scaler_C.transform(X_test_C)

In [ ]:
# Loop through different k values to see which has the highest accuracy
# train_scores = []
# test_scores = []
# for k in range(1, 20, 2):
#     knn = KNeighborsClassifier(n_neighbors=k)
#     knn.fit(X_train_scaled_C, y_resampled_C)
#     train_score = knn.score(X_train_scaled_C, y_resampled_C)
#     test_score = knn.score(X_test_scaled_C, y_test_C)
#     train_scores.append(train_score)
#     test_scores.append(test_score)
#     print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
# plt.plot(range(1, 20, 2), train_scores, marker='o')
# plt.plot(range(1, 20, 2), test_scores, marker="x")
# plt.xlabel("k neighbors")
# plt.ylabel("Testing accuracy Score")
# plt.show()

In [ ]:
#Run good K value

#Build Model
k=11
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train_scaled_C, y_resampled_C)

#Run Predictions
predictions_C = knn.predict(X_test_scaled_C)

#Run Scoring methods
score_C = knn.score(X_test_scaled_C, y_test_C)

probs = knn.predict_proba(X_test_scaled_C)[:, 1]
fpr, tpr, _ = roc_curve(y_test_C, probs)

precision_C, recall_C, thresholds_C = precision_recall_curve(y_test_C, probs)
auc_C = auc(recall_C, precision_C)

f1_C = f1_score(y_test_C, predictions_C)

values = [score_C, auc_C, f1_C]
score_df['KNN'] = values

precision_recall_df['Precision_KNN'] = precision_C
precision_recall_df['Recall_KNN'] = recall_C

roc_curve_df['fpr_KNN'] = fpr
roc_curve_df['tpr_KNN'] = tpr

precision_recall_KNN_df = pd.DataFrame({'Precision_KNN': precision_C, 'Recall_KNN': recall_C})
roc_curve_KNN_df = pd.DataFrame({'fpr_KNN': fpr, 'tpr_KNN':tpr})

predictions_df['KNN_Results'] = predictions_C

In [ ]:
#Run SVM

#Generate Model
model = SVC(kernel='linear')
model.fit(X_resampled_C, y_resampled_C)

#Run Predictions
predictions_C = model.predict(X_test_C)

#Run Scoring methods
score_C = SVC.score(X_test_C, y_test_C)

probs = SVC.predict_proba(X_test_C)[:, 1]
fpr, tpr, _ = roc_curve(y_test_C, probs)

precision_C, recall_C, thresholds_C = precision_recall_curve(y_test_C, probs)
auc_C = auc(recall_C, precision_C)

f1_C = f1_score(y_test_C, predictions_C)

values = [score_C, auc_C, f1_C]
score_df['SVC'] = values

precision_recall_SVM_df = pd.DataFrame({'Precision_SVM': precision_C, 'Recall_SVM': recall_C})
roc_curve_SVM_df = pd.DataFrame({'fpr_SVM': fpr, 'tpr_SVM':tpr})

predictions_df['SVC_Results'] = predictions_C 

In [ ]:
p = plot_confusion_matrix(model_log, X_test_C, y_test_C)

In [ ]:
p = plot_confusion_matrix(knn, X_test_scaled_C, y_test_C)

In [ ]:
p = plot_confusion_matrix(model, X_test_C, y_test_C)

In [ ]:
score_df.to_csv('Resources/ml_scores_synthetic.csv')

precision_recall_log_df.to_csv('Resources/ml_precision_recall_LOG_synthetic.csv')
precision_recall_SVM_df.to_csv('Resources/ml_precision_recall_SVM_synthetic.csv')
precision_recall_KNN_df.to_csv('Resources/ml_precision_recall_KNN_synthetic.csv')
roc_curve_log_df.to_csv('Resources/ml_roc_LOG_synthetic.csv')
roc_curve_SVM_df.to_csv('Resources/ml_roc_SVM_synthetic.csv')
roc_curve_KNN_df.to_csv('Resources/ml_roc_KNN_synthetic.csv')

predictions_df.to_csv('Resources/ml_predictions_synthetic.csv')